In [1]:
from tqdm import tqdm
import pandas as pd
import cv2

from common.metrics import calculate_ssim, calculate_psnr, calculate_lpips, calculate_gmsd
from common.config import BLUR_PARAM_RANGES

from pathlib import Path

def run_metrics(
        dataset_path: Path,
        original_path: Path,
        deblurred_path: Path,
        save: bool = False
):
    df = pd.read_parquet(dataset_path)

    for blur_type in BLUR_PARAM_RANGES.keys():
        for key in tqdm(df.dropna()['key'], desc=f"Metrics: {blur_type:<8}\t"):
            img1_path = original_path / f"{key}.png"
            img2_path = deblurred_path / blur_type / f"{key}.png"

            img1 = cv2.imread(img1_path)
            img2 = cv2.imread(img2_path)

            ssim = calculate_ssim(img1, img2)
            psnr = calculate_psnr(img1, img2)
            lpips = calculate_lpips(img1, img2)
            gmsd = calculate_gmsd(img1, img2)

            df.loc[df['key'] == key, f'ssim_{blur_type}'] = ssim
            df.loc[df['key'] == key, f'psnr_{blur_type}'] = psnr
            df.loc[df['key'] == key, f'lpips_{blur_type}'] = lpips
            df.loc[df['key'] == key, f'gmsd_{blur_type}'] = gmsd
    
    if save:
        df.to_parquet(dataset_path)
    
    return df

In [2]:
from common.config import TEST_ORIGINAL_DIR, TEST_DEBLURRED_DIR, IDPA_DIR

name = "pseudoinverse_nokernel"

run_metrics(
    dataset_path=IDPA_DIR / f"{name}.parquet",
    original_path=TEST_ORIGINAL_DIR / "00000",
    deblurred_path=TEST_DEBLURRED_DIR / name,
    save=True
)

Metrics: motion  	: 100%|██████████| 950/950 [19:33<00:00,  1.23s/it]


,key,proctime_box,proctime_gaussian,proctime_motion,ssim_box,psnr_box,lpips_box,gmsd_box,ssim_gaussian,psnr_gaussian,lpips_gaussian,gmsd_gaussian,ssim_motion,psnr_motion,lpips_motion,gmsd_motion
2,000000382,0.556124,0.552079,0.560412,0.321270,16.805201,0.104542,0.258854,0.120108,14.031477,0.043545,0.321564,0.043244,9.750979,0.223020,0.306803
5,000000931,0.421191,0.419491,0.424831,0.041470,10.657864,0.122841,0.311788,0.027412,8.452177,0.249487,0.296981,0.021377,8.004143,0.358679,0.322604
6,000000558,0.699464,0.709690,0.695441,0.436767,17.665337,0.111135,0.218454,0.268639,15.220823,0.044039,0.240499,0.490566,16.274383,0.073231,0.234188
8,000001226,1.045535,1.108337,1.041143,0.417841,15.581724,0.147482,0.300871,0.138062,11.764239,0.081711,0.355220,0.077170,9.723992,0.385124,0.318856
9,000000225,0.641634,0.642698,0.644093,0.265539,18.716036,0.078107,0.245702,0.033799,9.568917,0.395121,0.272990,0.159682,13.903596,0.117832,0.291392
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1245,000000564,0.711418,0.707743,0.710086,0.179187,13.625821,0.078923,0.314733,0.080389,8.506589,0.311508,0.342172,0.098402,9.242675,0.158240,0.321157
1246,000001225,1.576768,1.648128,1.810443,0.255719,15.332080,0.141714,0.248644,0.005228,6.174511,0.461914,0.336666,0.032262,7.328061,0.320486,0.326655
1247,000000481,0.679964,0.669008,0.706785,0.119590,14.587961,0.041404,0.301542,0.290917,18.223066,0.032298,0.244371,0.065441,9.373487,0.223518,0.301148
1248,000000954,0.383162,0.383533,0.386578,0.185581,15.145490,0.165229,0.305571,0.195834,16.155014,0.147223,0.292313,0.074794,12.070900,0.103442,0.312394


In [6]:
from common.config import TEST_ORIGINAL_DIR, TEST_DEBLURRED_DIR, IDPA_DIR

name = "pseudoinverse_wkernel"

run_metrics(
    dataset_path=IDPA_DIR / f"{name}.parquet",
    original_path=TEST_ORIGINAL_DIR / "00000",
    deblurred_path=TEST_DEBLURRED_DIR / name,
    save=True
)

Metrics: motion  	: 100%|██████████| 950/950 [19:44<00:00,  1.25s/it]


,key,proctime_box,proctime_gaussian,proctime_motion,ssim_box,psnr_box,lpips_box,gmsd_box,ssim_gaussian,psnr_gaussian,lpips_gaussian,gmsd_gaussian,ssim_motion,psnr_motion,lpips_motion,gmsd_motion
0,000000291,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,000000987,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,000000382,0.556124,0.552079,0.560412,0.205649,12.652392,0.075706,0.281280,0.338098,12.681865,0.057393,0.312401,0.231287,12.772802,0.053236,0.298083
3,000000058,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,000000001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1245,000000564,0.711418,0.707743,0.710086,0.148042,12.123445,0.169734,0.309215,0.352196,13.744316,0.029493,0.292137,0.178644,13.532860,0.031328,0.279417
1246,000001225,1.576768,1.648128,1.810443,0.145848,12.719726,0.092198,0.263764,0.278534,13.445715,0.032391,0.256270,0.139884,12.351754,0.085434,0.267706
1247,000000481,0.679964,0.669008,0.706785,0.224264,15.946854,0.036571,0.240665,0.302581,15.570594,0.061666,0.267258,0.241123,15.311467,0.045686,0.246421
1248,000000954,0.383162,0.383533,0.386578,0.057137,12.494349,0.463937,0.308344,0.120435,12.010087,0.156605,0.309401,0.125911,13.751603,0.086925,0.292354


In [2]:
from common.config import TEST_ORIGINAL_DIR, TEST_DEBLURRED_DIR, IDPA_DIR

name = "wiener_nokernel"

run_metrics(
    dataset_path=IDPA_DIR / f"{name}.parquet",
    original_path=TEST_ORIGINAL_DIR / "00000",
    deblurred_path=TEST_DEBLURRED_DIR / name,
    save=True
)

Metrics: motion  	: 100%|██████████| 950/950 [20:37<00:00,  1.30s/it]


,key,proctime_box,proctime_gaussian,proctime_motion,ssim_box,psnr_box,lpips_box,gmsd_box,ssim_gaussian,psnr_gaussian,lpips_gaussian,gmsd_gaussian,ssim_motion,psnr_motion,lpips_motion,gmsd_motion
0,000000000,0.458832,0.515758,0.462619,0.496839,18.977943,0.126385,0.249297,0.468064,18.702694,0.093637,0.284283,0.334376,13.861960,0.068649,0.312474
1,000000002,0.457824,0.478058,0.460747,0.184690,15.485043,0.072707,0.241003,0.162212,15.053017,0.079676,0.242625,0.093464,13.648773,0.158376,0.273850
2,000000003,0.615560,0.606898,0.601368,0.105265,14.587455,0.077475,0.300390,0.395504,19.595360,0.154698,0.260329,0.185484,15.149970,0.226845,0.299215
3,000000004,0.427569,0.416815,0.414729,0.315277,15.306178,0.231524,0.272371,0.266199,14.633955,0.102267,0.287295,0.164425,12.198738,0.269923,0.294317
4,000000005,0.571736,0.559716,0.558967,0.353146,17.785113,0.162414,0.258377,0.183185,11.047638,0.226223,0.301999,0.251204,15.311221,0.163472,0.253726
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
945,000001245,0.935081,0.938367,0.921489,0.266700,11.880445,0.036817,0.316571,0.382016,13.020934,0.071874,0.332916,0.217987,7.915343,0.178809,0.373328
946,000001246,1.110227,1.123377,1.078878,0.320874,17.423777,0.289820,0.271511,0.307444,17.347380,0.152111,0.310329,0.108402,11.436147,0.181782,0.303954
947,000001247,1.188854,1.202173,1.159555,0.221787,14.975796,0.071808,0.279396,0.203191,14.047853,0.048440,0.272331,0.100915,10.854823,0.254368,0.317027
948,000001248,0.644407,0.651523,0.633614,0.315127,16.406077,0.289839,0.258532,0.214891,13.571545,0.052846,0.273099,0.125721,9.701543,0.195918,0.313943


In [2]:
from common.config import TEST_ORIGINAL_DIR, TEST_DEBLURRED_DIR, IDPA_DIR

name = "wiener_wkernel"

run_metrics(
    dataset_path=IDPA_DIR / f"{name}.parquet",
    original_path=TEST_ORIGINAL_DIR / "00000",
    deblurred_path=TEST_DEBLURRED_DIR / name,
    save=True
)

Metrics: motion  	: 100%|██████████| 950/950 [20:38<00:00,  1.30s/it]


,key,proctime_box,proctime_gaussian,proctime_motion,ssim_box,psnr_box,lpips_box,gmsd_box,ssim_gaussian,psnr_gaussian,lpips_gaussian,gmsd_gaussian,ssim_motion,psnr_motion,lpips_motion,gmsd_motion
0,000000000,0.420980,0.410270,0.403221,0.306041,15.238008,0.150376,0.273557,0.397968,14.216881,0.087464,0.288205,0.418671,17.574850,0.026399,0.259695
1,000000002,0.411562,0.410581,0.398916,0.237640,17.385971,0.040085,0.226769,0.212007,15.601599,0.073179,0.236719,0.153227,14.098981,0.178854,0.243117
2,000000003,0.547179,0.555004,0.546046,0.311391,18.361787,0.026017,0.240287,0.277466,14.834785,0.128543,0.267691,0.179752,14.030523,0.160408,0.264773
3,000000004,0.379000,0.389913,0.378511,0.128901,11.447377,0.135286,0.291385,0.194965,11.101231,0.130154,0.307901,0.088882,10.369691,0.173964,0.296527
4,000000005,0.508115,0.508992,0.502674,0.269944,15.103669,0.119628,0.254166,0.256612,15.244626,0.036341,0.267358,0.157107,13.777706,0.127133,0.264536
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
945,000001245,0.840535,0.852951,0.842899,0.305956,11.254302,0.043582,0.320323,0.307762,9.788058,0.106114,0.339443,0.319283,10.758772,0.043181,0.329697
946,000001246,1.020220,1.022607,1.014991,0.205378,14.808251,0.116365,0.268331,0.245717,14.671553,0.120742,0.280851,0.180119,14.109092,0.102178,0.268523
947,000001247,1.091541,1.080063,1.088065,0.201129,13.892894,0.058040,0.275251,0.233687,13.616024,0.076219,0.292044,0.132145,12.975692,0.106660,0.289410
948,000001248,0.601176,0.619459,0.597491,0.213827,14.290957,0.078695,0.269415,0.247722,14.006288,0.061805,0.274811,0.212478,13.756304,0.053219,0.260392


In [3]:
from common.config import TEST_ORIGINAL_DIR, TEST_DEBLURRED_DIR, IDPA_DIR

name = "deblurganv2"

run_metrics(
    dataset_path=IDPA_DIR / f"{name}.parquet",
    original_path=TEST_ORIGINAL_DIR / "00000",
    deblurred_path=TEST_DEBLURRED_DIR / name,
    save=True
)

Metrics: motion  	: 100%|██████████| 950/950 [20:36<00:00,  1.30s/it]


,key,proctime_box,proctime_gaussian,proctime_motion,ssim_box,psnr_box,lpips_box,gmsd_box,ssim_gaussian,psnr_gaussian,lpips_gaussian,gmsd_gaussian,ssim_motion,psnr_motion,lpips_motion,gmsd_motion
0,000000000,0.771006,0.227473,0.227028,0.624878,21.736350,0.063853,0.269811,0.618547,21.525129,0.089766,0.278274,0.789718,24.130607,0.005149,0.174958
1,000000002,0.243786,0.229896,0.233648,0.592647,22.730560,0.006872,0.227930,0.655491,23.634710,0.010482,0.216396,0.536058,21.578249,0.040192,0.224285
2,000000003,0.220997,0.182720,0.180572,0.716148,25.098564,0.004474,0.189143,0.540464,21.949155,0.144286,0.256134,0.667576,23.922039,0.035748,0.192016
3,000000004,0.138043,0.125477,0.126331,0.431818,16.745145,0.201834,0.314505,0.506818,17.644235,0.136575,0.294406,0.585895,18.283055,0.046525,0.244140
4,000000005,0.189678,0.190147,0.189680,0.402609,18.877857,0.187486,0.330045,0.744844,22.947446,0.003723,0.159097,0.505906,19.817103,0.053871,0.260503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
945,000001245,0.367004,0.366534,0.361780,0.532681,15.147850,0.008479,0.303925,0.447492,14.320671,0.052568,0.329849,0.605614,15.558908,0.005707,0.268189
946,000001246,0.245533,0.238348,0.240876,0.372599,17.904166,0.248901,0.284748,0.427196,18.529201,0.156812,0.277338,0.585976,19.503085,0.046899,0.212796
947,000001247,0.312241,0.317050,0.322241,0.437299,17.389728,0.054384,0.305659,0.582381,18.770591,0.019877,0.275850,0.543111,17.840805,0.038324,0.238710
948,000001248,0.200348,0.203363,0.201590,0.351374,16.865351,0.245513,0.314531,0.570243,18.832031,0.017088,0.238377,0.577636,18.229012,0.005415,0.212806
